# Imports

In [2]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import sys
import dask as dd
from dask import dataframe
sys.path.insert(0, "C:/workspace/stress_test_sim//libs")

from monte_carlo import CauchyGenerator, PoissonGenerator, NormalGenerator, ParetoGenerator, LognormalGenerator, Transaction, MonteCarloTransactionSimulator, CauchyParameterSearcher, LognormalParameterSearcher
from uniswap_v2_extractor import get_pool_v2_reserves_history, get_pool_v2_history, list_to_transaction_dictionary, list_to_reserves_dictionary, pool_reserves_to_df, pool_history_to_df, get_pool_v2_mints, get_pool_v2_burns, list_to_mints_dictionary, pool_mints_to_df, pool_burns_to_df
from dima_trubca_v2_plots import show_swaps_count_moving_averages, show_swaps_reserves_evolution_through_time, show_pool_price_evolution_from_reserves, show_swaps_amount_in_moving_averages

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# WETH/USDC pool data

## Collecting WETH/USDC mints

In [6]:
eth_usdc_mints = get_pool_v2_mints(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=33)
eth_usdc_mints_df = pool_mints_to_df(eth_usdc_mints)

100%|██████████| 33/33 [00:45<00:00,  1.39s/it]


### writing received data to the pickle

In [9]:
eth_usdc_mints_df.to_pickle(os.getcwd() + "\\eth_usdc_mints.pkl")

### reading received data from pickle

In [ ]:
eth_usdc_mints_df = pd.read_pickle(os.getcwd() + "\\eth_usdc_mints.pkl")

## Collecting WETH/USDC burns

In [17]:
eth_usdc_burns = get_pool_v2_burns(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=27)
eth_usdc_burns_df = pool_burns_to_df(eth_usdc_burns)

 93%|█████████▎| 25/27 [00:30<00:02,  1.06s/it]

list index out of range


 96%|█████████▋| 26/27 [00:30<00:00,  1.21it/s]

list index out of range


100%|██████████| 27/27 [00:30<00:00,  1.15s/it]

list index out of range


In [18]:
eth_usdc_burns_df

,amount0,amount1,amountUSD,liquidity,timestamp
0,3.262000e-03,0.000016,3.263003e-03,2.299449e-10,2020-05-07 15:47:25
1,1.418900e-02,0.000070,1.419001e-02,9.999739e-10,2020-05-08 02:43:59
2,7.812840e-01,0.006539,7.812847e-01,6.960485e-08,2020-05-12 20:18:50
3,9.953090e-01,0.004114,8.361725e-01,6.217433e-08,2020-05-13 23:48:21
4,1.097105e+00,0.005132,1.047697e+00,7.289922e-08,2020-05-14 14:24:41
...,...,...,...,...,...
23307,2.613006e+03,0.543883,5.227160e+03,2.112073e-05,2021-11-09 10:28:54
23308,3.257345e+05,69.085029,6.515458e+05,2.656415e-03,2021-11-10 01:06:40
23309,1.090435e+06,229.948711,2.180152e+06,8.866823e-03,2021-11-10 02:42:27
23310,1.573420e+02,0.033234,3.146902e+02,1.280273e-06,2021-11-10 10:59:39


### Writing received data to pickle

In [19]:
eth_usdc_burns_df.to_pickle(os.getcwd() + "\\eth_usdc_burns.pkl")

### Reading received data from pickle

In [21]:
eth_usdc_burns_df = pd.read_pickle(os.getcwd() + "\\eth_usdc_burns.pkl")

## Plotting burns data

# WBTC/USDC pool data

## Collecting WBTC/USDC swaps, mints, burns, reserves

In [ ]:
wbtc_usdc_history = get_pool_v2_history(contract_id='0x004375dff511095cc5a197a54140a24efef3a416', range_limit=100)
wbtc_usdc_df = pool_history_to_df(wbtc_usdc_history)
wbtc_usdc_df

In [ ]:
wbtc_usdc_mints = get_pool_v2_mints(contract_id='0x004375dff511095cc5a197a54140a24efef3a416', range_limit=2)
wbtc_usdc_mints_df = pool_mints_to_df(wbtc_usdc_mints)
wbtc_usdc_mints_df

In [ ]:
wbtc_usdc_burns = get_pool_v2_burns(contract_id='0x004375dff511095cc5a197a54140a24efef3a416', range_limit=2)
wbtc_usdc_burns_df = pool_burns_to_df(wbtc_usdc_burns)
wbtc_usdc_burns_df

In [ ]:
wbtc_usdc_reserves = get_pool_v2_reserves_history(contract_id='0x004375dff511095cc5a197a54140a24efef3a416')
wbtc_usdc_reserves_df = pool_reserves_to_df(wbtc_usdc_reserves)
wbtc_usdc_reserves_df

### Writing swaps, mints, burns, reserves into pickle files

In [58]:
wbtc_usdc_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_swaps.pkl")
wbtc_usdc_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_mints.pkl")
wbtc_usdc_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_burns.pkl")
wbtc_usdc_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_reserves.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [60]:
wbtc_usdc_df = pd.read_pickle(os.getcwd() + '\\pkl_stories\\wbtc_usdc_swaps.pkl')
wbtc_usdc_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_mints.pkl")
wbtc_usdc_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_burns.pkl")
wbtc_usdc_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_reserves.pkl")

# WBTC/DAI pool data

## Collecting swaps, mints, burns and reserves

In [47]:
wbtc_dai_history = get_pool_v2_history(contract_id='0x231b7589426ffe1b75405526fc32ac09d44364c4', range_limit=20)
wbtc_dai_df = pool_history_to_df(wbtc_dai_history)
wbtc_dai_df

 80%|████████  | 16/20 [00:21<00:02,  1.39it/s]

list index out of range
list index out of range


 90%|█████████ | 18/20 [00:22<00:00,  2.20it/s]

list index out of range
list index out of range


 95%|█████████▌| 19/20 [00:22<00:00,  2.49it/s]

list index out of range


100%|██████████| 20/20 [00:22<00:00,  1.14s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,WBTC,DAI,0.000039,0.378642,0.378116,2020-05-20 11:55:36
1,WBTC,DAI,0.000017,0.162868,0.161100,2020-05-20 18:34:15
2,WBTC,DAI,0.000256,2.450812,2.382097,2020-05-22 00:32:08
3,WBTC,DAI,0.000022,0.208767,0.206079,2020-05-22 10:27:44
4,WBTC,DAI,0.000022,0.208199,0.205794,2020-05-22 10:30:29
...,...,...,...,...,...,...
13722,DAI,WBTC,29.994001,0.000448,29.982305,2021-11-10 14:10:02
13723,DAI,WBTC,4563.590414,0.067734,4564.780584,2021-11-10 14:11:41
13724,DAI,WBTC,2050.122916,0.030134,2049.562041,2021-11-10 14:27:29
13725,DAI,WBTC,333.128567,0.004880,332.528501,2021-11-10 14:51:37


In [49]:
wbtc_dai_reserves = get_pool_v2_reserves_history(contract_id='0x231b7589426ffe1b75405526fc32ac09d44364c4')
wbtc_dai_reserves_df = pool_reserves_to_df(wbtc_dai_reserves)
wbtc_dai_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.48it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,0.015897,156.421121,3.110206e+02,0.000000,0.000000,2020-05-18
1,0.015952,155.879611,3.080758e+02,0.000055,0.541510,2020-05-20
2,0.116257,1093.860346,2.163052e+03,0.000305,2.916207,2020-05-22
3,0.117786,1079.701885,2.147687e+03,0.001529,14.158461,2020-05-23
4,0.120951,1051.538677,2.108504e+03,0.003164,28.163208,2020-05-24
...,...,...,...,...,...,...
525,10.605367,646047.507200,1.297103e+06,0.028831,1754.970336,2021-11-06
526,10.463013,654892.635091,1.308431e+06,0.296864,18390.629186,2021-11-07
527,10.136547,676064.119752,1.355040e+06,0.402897,26167.950680,2021-11-08
528,10.136898,676139.780215,1.354080e+06,0.492093,33316.639694,2021-11-09


In [53]:
wbtc_dai_mints = get_pool_v2_mints(contract_id='0x231b7589426ffe1b75405526fc32ac09d44364c4', range_limit=1)
wbtc_dai_mints_df = pool_mints_to_df(wbtc_dai_mints)
wbtc_dai_mints_df

100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.000058,0.573588,1.138998,5.782254e-08,2020-05-18 21:18:56
1,0.015859,156.045929,310.274573,1.573075e-05,2020-05-18 22:22:40
2,0.100000,940.896942,1860.574893,9.699443e-05,2020-05-22 21:37:08
3,0.008735,77.283604,154.781806,8.215019e-06,2020-05-26 12:22:59
4,0.056679,501.500211,1000.979806,5.330799e-05,2020-05-26 22:37:46
...,...,...,...,...,...
180,2.426354,144299.176047,288573.318661,5.448097e-03,2021-10-15 12:43:52
181,0.742799,45843.142887,91775.017482,1.698149e-03,2021-10-16 05:27:11
182,1.225796,75652.079257,151603.780405,2.802349e-03,2021-10-16 05:47:00
183,1.434042,88504.336796,177438.237016,3.278430e-03,2021-10-16 06:03:22


In [54]:
wbtc_dai_burns = get_pool_v2_burns(contract_id='0x231b7589426ffe1b75405526fc32ac09d44364c4', range_limit=1)
wbtc_dai_burns_df = pool_burns_to_df(wbtc_dai_burns)
wbtc_dai_burns_df

100%|██████████| 1/1 [00:00<00:00,  1.95it/s]


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.000020,0.198396,0.393071,2.000000e-08,2020-05-18 21:23:57
1,0.100134,940.023349,1879.119093,9.699443e-05,2020-05-29 21:32:50
2,0.164053,1560.514491,3113.745579,1.599430e-04,2020-06-01 12:07:18
3,0.110400,1095.423005,2144.658189,1.098884e-04,2020-06-11 15:32:31
4,0.205953,1835.102135,3700.348283,1.942387e-04,2020-06-15 14:15:28
...,...,...,...,...,...
131,3.082148,154638.721350,309660.663023,6.382034e-03,2021-09-04 10:38:20
132,1.180265,60639.818523,121701.328399,2.472910e-03,2021-09-05 23:24:45
133,0.080422,3671.205894,7354.563725,1.587508e-04,2021-09-11 18:06:20
134,6.832808,328088.100764,656524.081660,1.380124e-02,2021-10-02 17:14:22


### Writing swaps, mints, burns, reserves into pickle files

In [61]:
wbtc_dai_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_swaps.pkl")
wbtc_dai_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_reserves.pkl")
wbtc_dai_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_mints.pkl")
wbtc_dai_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [57]:
wbtc_dai_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_swaps.pkl")
wbtc_dai_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_reserves.pkl")
wbtc_dai_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_mints.pkl")
wbtc_dai_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_burns.pkl")

# IXS/ETH pool information

## Getting IXS/ETH swaps, mints, burns, reserves data

In [63]:
ixs_weth_history = get_pool_v2_history(contract_id='0xc09bf2b1bc8725903c509e8caeef9190857215a8', range_limit=4)
ixs_weth_df = pool_history_to_df(ixs_weth_history)
ixs_weth_df

 75%|███████▌  | 3/4 [00:03<00:00,  1.13it/s]

list index out of range


100%|██████████| 4/4 [00:03<00:00,  1.19it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,WETH,IXS,12.000000,211315.866941,49367.994152,2021-09-08 12:59:29
1,IXS,WETH,10781.086133,1.010521,3412.553806,2021-09-08 12:59:42
2,WETH,IXS,0.886100,10781.086133,3050.396683,2021-09-08 12:59:42
3,WETH,IXS,3.000000,33333.139648,10583.791945,2021-09-08 12:59:42
4,WETH,IXS,0.146153,1580.639685,498.586381,2021-09-08 12:59:58
...,...,...,...,...,...,...
1872,WETH,IXS,1.717427,19682.847766,8105.747852,2021-11-10 10:21:01
1873,IXS,WETH,15274.361563,1.329625,6294.260721,2021-11-10 11:47:29
1874,WETH,IXS,0.700000,8040.665949,3312.960006,2021-11-10 12:27:44
1875,IXS,WETH,791.286183,0.068881,334.219999,2021-11-10 14:22:37


In [65]:
ixs_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0xc09bf2b1bc8725903c509e8caeef9190857215a8')
ixs_weth_reserves_df = pool_reserves_to_df(ixs_weth_reserves_history)
ixs_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  4.36it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,3.794422e+05,55.944001,3.907889e+05,3.245787e+06,425.302832,2021-09-08
1,3.880956e+05,54.966786,3.762270e+05,6.385440e+05,93.661413,2021-09-09
2,3.895395e+05,54.992914,3.521057e+05,5.129601e+05,85.994939,2021-09-10
3,3.251391e+05,66.178741,4.314507e+05,5.158716e+05,94.863760,2021-09-11
4,3.138946e+05,68.793197,4.722017e+05,3.769885e+05,81.732506,2021-09-12
...,...,...,...,...,...,...
59,1.252272e+06,111.369945,9.959401e+05,1.743100e+05,16.011554,2021-11-06
60,1.262068e+06,103.855905,9.562487e+05,1.157668e+05,9.582805,2021-11-07
61,1.300168e+06,100.830539,9.533918e+05,7.774344e+04,6.108676,2021-11-08
62,1.280204e+06,102.587502,9.790004e+05,1.272239e+05,9.640691,2021-11-09


In [67]:
ixs_weth_mints_history = get_pool_v2_mints(contract_id='0xc09bf2b1bc8725903c509e8caeef9190857215a8', range_limit=2)
ixs_weth_mints_df = pool_mints_to_df(ixs_weth_mints_history)
ixs_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  3.21it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,720000.000000,28.800000,98227.795887,4553.679831,2021-09-08 12:59:29
1,1360.163496,0.195028,1414.289538,15.913010,2021-09-16 13:03:24
2,14999.994987,2.150790,15598.796295,175.489987,2021-09-16 13:04:20
3,10461.268874,1.500000,10880.913058,122.389903,2021-09-16 13:06:30
4,3010.000000,0.412264,2993.141508,34.416269,2021-09-16 13:12:28
...,...,...,...,...,...
110,197.869987,0.017645,152.861486,1.804954,2021-10-29 07:17:25
111,16166.492978,1.950000,17012.881531,171.196836,2021-11-01 14:01:52
112,1428.439848,0.146827,1315.281150,13.959495,2021-11-02 14:16:58
113,3293.122422,0.306428,2777.935907,30.615115,2021-11-03 17:13:47


In [68]:
ixs_weth_burns_history = get_pool_v2_burns(contract_id='0xc09bf2b1bc8725903c509e8caeef9190857215a8', range_limit=2)
ixs_weth_burns_df = pool_burns_to_df(ixs_weth_burns_history)
ixs_weth_burns_df.head(5)

100%|██████████| 2/2 [00:00<00:00,  3.34it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,635.707371,0.086897,630.742972,7.261432,2021-09-16 13:25:29
1,7.493118,0.000959,6.934917,0.082742,2021-09-16 16:44:04
2,3169.601280,0.405602,2929.932964,35.000000,2021-09-16 16:55:51
3,2160.670975,0.285767,2034.020295,24.254484,2021-09-16 18:41:38
4,644.837670,0.085285,609.778679,7.238587,2021-09-16 19:40:37
5,10922.626248,1.439562,10210.884110,122.389903,2021-09-16 22:01:17
6,48689.119120,5.783553,41138.322816,517.896918,2021-09-17 03:00:24
7,6591.566272,0.735324,5243.750576,67.937636,2021-09-17 03:57:34
8,1209.244633,0.143953,1024.904868,12.874108,2021-09-17 06:05:17
9,7601.550942,0.888382,5994.633130,80.171938,2021-09-17 18:39:49


### Saving swaps, mints, burns, reserves into pickle files

In [69]:
ixs_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_swaps.pkl")
ixs_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_reserves.pkl")
ixs_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_mints.pkl")
ixs_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_burns.pkl")

### Loading swaps, mints, burns, reserves from pickle files

In [70]:
ixs_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_swaps.pkl")
ixs_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_reserves.pkl")
ixs_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_mints.pkl")
ixs_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_burns.pkl")

# SQUID/ETH pool 

It is important to mention that this pool was chosen considering that it became a part of a fraud scheme performed by the creators of the SQUID token and it will be interesting to go through this story, run simulation over it.

In [71]:
squid_weth_history = get_pool_v2_history(contract_id='0x40df4eb23d23bb7af5c6bb7bc9a185913d97e7b9', range_limit=4)
squid_weth_df = pool_history_to_df(squid_weth_history)
squid_weth_df

 75%|███████▌  | 3/4 [00:01<00:00,  3.31it/s]

list index out of range
list index out of range


100%|██████████| 4/4 [00:01<00:00,  2.31it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,SQUID,WETH,1.309912e+08,0.0,277.241444,2021-09-29 21:57:29
1,SQUID,WETH,2.621781e+07,0.0,63.787799,2021-09-29 21:57:29
2,SQUID,WETH,8.137033e+07,0.0,85.409522,2021-09-29 21:57:29
3,SQUID,WETH,1.814679e+08,0.0,144.731390,2021-09-29 21:57:29
4,SQUID,WETH,1.265292e+08,0.0,297.044218,2021-09-29 21:57:29
...,...,...,...,...,...,...
322,SQUID,WETH,1.631955e+08,0.0,227.617574,2021-10-20 16:03:10
323,SQUID,WETH,1.387422e+08,0.0,208.924346,2021-10-22 02:28:51
324,SQUID,WETH,6.648252e+07,0.0,108.827934,2021-10-29 11:43:53
325,SQUID,WETH,5.962051e+07,0.0,100.410878,2021-10-29 13:11:53


In [73]:
squid_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x40df4eb23d23bb7af5c6bb7bc9a185913d97e7b9')
squid_weth_reserves_df = pool_reserves_to_df(squid_weth_reserves_history)
squid_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  3.76it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,4.844201e+11,2.705857,15381.062606,1.976699e+12,38.460760,2021-09-29
1,5.625445e+11,2.336738,14192.247090,2.373009e+11,1.003119,2021-09-30
2,5.699927e+11,2.306294,15185.894105,7.448186e+09,0.030444,2021-10-02
3,5.686125e+11,2.311974,16624.768469,1.411990e+09,0.005680,2021-10-07
4,5.682726e+11,2.314200,16394.571671,2.366301e+10,0.097774,2021-10-10
5,5.790447e+11,2.271275,15513.444104,1.077210e+10,0.042925,2021-10-12
6,5.773171e+11,2.278172,17561.696928,1.767880e+09,0.006897,2021-10-17
7,5.900352e+11,2.229211,17237.288891,1.271807e+10,0.048961,2021-10-18
8,5.760329e+11,2.284211,18598.841009,1.432866e+10,0.055000,2021-10-20
9,5.638690e+11,2.334211,19218.093319,1.244144e+10,0.050000,2021-10-22


In [76]:
squid_weth_mints_history = get_pool_v2_mints(contract_id='0x40df4eb23d23bb7af5c6bb7bc9a185913d97e7b9', range_limit=2)
squid_weth_mints_df = pool_mints_to_df(squid_weth_mints_history)
squid_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  2.47it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,7.381000e+11,1.7,4788.810094,35.422733,2021-09-29 21:57:29


There are no burns in the pool

In [78]:
squid_weth_burns_history = get_pool_v2_burns(contract_id='0x40df4eb23d23bb7af5c6bb7bc9a185913d97e7b9', range_limit=1)
squid_weth_burns_df = pool_burns_to_df(squid_weth_burns_history)
squid_weth_burns_df

100%|██████████| 1/1 [00:00<00:00,  2.64it/s]

list index out of range


AttributeError: 'DataFrame' object has no attribute 'amount0'

### Writing swaps, mints and reserves into pickle files

In [79]:
squid_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_swaps.pkl")
squid_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_reserves.pkl")
squid_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_mints.pkl")

### Reading swaps, mints and reserves from pickle files

In [80]:
squid_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_swaps.pkl")
squid_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_reserves.pkl")
squid_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_mints.pkl")

# HKMT/USDT pool

In [82]:
hkmt_usdt_history = get_pool_v2_history(contract_id='0xf52f433b79d21023af94251958bed3b64a2b7930', range_limit=1)
hkmt_usdt_df = pool_history_to_df(hkmt_usdt_history)
hkmt_usdt_df

100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,USDT,HKMT,1388.137176,1.072529e+04,1377.513809,2020-09-01 09:06:15
1,USDT,HKMT,2315.057158,1.788484e+04,2296.644225,2020-09-01 09:07:04
2,HKMT,USDT,10725.294140,1.380034e+03,1369.211622,2020-09-01 09:07:36
3,USDT,HKMT,42.510850,3.284195e+02,42.275994,2020-09-01 09:25:52
4,HKMT,USDT,17884.839162,2.300983e+03,2293.554386,2020-09-01 09:32:21
...,...,...,...,...,...,...
330,HKMT,USDT,21000.000000,2.708539e+03,2703.402125,2021-10-20 09:24:33
331,HKMT,USDT,1250.000000,1.612133e+02,161.163034,2021-10-20 09:39:23
332,USDT,HKMT,149976.000000,1.152465e+06,149883.280701,2021-10-22 05:29:12
333,HKMT,USDT,10000.000000,1.297395e+03,1295.051760,2021-11-09 12:54:07


In [84]:
hkmt_usdt_reserves_history = get_pool_v2_reserves_history(contract_id='0xf52f433b79d21023af94251958bed3b64a2b7930')
hkmt_usdt_reserves_df = pool_reserves_to_df(hkmt_usdt_reserves_history)
hkmt_usdt_reserves_df

100%|██████████| 2/2 [00:01<00:00,  1.18it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,2.324374e+08,3.000841e+07,5.962596e+07,8.411800e+05,108718.139712,2020-09-01
1,2.324720e+08,3.000396e+07,5.998884e+07,3.968598e+04,5109.376059,2020-09-02
2,2.324620e+08,3.000526e+07,5.980636e+07,1.213620e+04,1570.298902,2020-09-03
3,2.324728e+08,3.000387e+07,5.974688e+07,1.126615e+04,1449.936710,2020-09-04
4,2.324795e+08,3.000300e+07,5.981387e+07,7.698807e+03,990.995574,2020-09-05
...,...,...,...,...,...,...
149,3.876795e+08,4.998382e+07,9.987336e+07,1.000000e+04,1285.473001,2021-10-19
150,3.870407e+08,5.006692e+07,1.001026e+08,1.566792e+06,202352.471319,2021-10-20
151,3.858882e+08,5.021690e+07,1.003717e+08,1.152465e+06,149976.000000,2021-10-22
152,3.858982e+08,5.021560e+07,1.002498e+08,1.000000e+04,1297.395174,2021-11-09


In [86]:
hkmt_usdt_mints_history = get_pool_v2_mints(contract_id='0xf52f433b79d21023af94251958bed3b64a2b7930', range_limit=2)
hkmt_usdt_mints_df = pool_mints_to_df(hkmt_usdt_mints_history)
hkmt_usdt_mints_df

100%|██████████| 2/2 [00:00<00:00,  3.86it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,2.325000e+08,3.000000e+07,2.980819e+07,8.351647e-05,2020-09-01 09:05:07
1,1.549714e+08,2.000000e+07,3.987585e+07,5.567218e-05,2020-09-07 06:32:58
2,1.435528e+02,1.852842e+01,3.681437e+01,5.157298e-11,2020-09-14 14:16:14
3,1.500000e+02,1.936058e+01,3.852151e+01,5.388924e-11,2020-09-14 17:06:08
4,5.760610e+02,7.435276e+01,1.478349e+02,2.069570e-10,2020-09-14 18:46:57
5,3.089770e+02,3.988003e+01,7.934419e+01,1.110039e-10,2020-09-15 01:41:11
6,2.897847e+01,3.740286e+00,7.427972e+00,1.041088e-11,2020-09-15 04:43:57
7,2.915278e+02,3.762785e+01,7.489703e+01,1.047350e-10,2020-09-15 12:42:05
8,1.000000e+01,1.290712e+00,2.567839e+00,3.592625e-12,2020-09-15 12:50:36
9,2.905769e+03,3.750568e+02,7.477305e+02,1.043942e-09,2020-09-15 12:57:14


In [87]:
hkmt_usdt_burns_history = get_pool_v2_burns(contract_id='0xf52f433b79d21023af94251958bed3b64a2b7930', range_limit=2)
hkmt_usdt_burns_df = pool_burns_to_df(hkmt_usdt_burns_history)
hkmt_usdt_burns_df

100%|██████████| 2/2 [00:00<00:00,  4.37it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,5665.201219,731.338390,1460.247116,2.035467e-09,2020-09-16 16:40:02
1,5684.618907,733.929268,1466.409881,2.042560e-09,2020-09-16 18:57:59
2,27801.582229,3589.298650,7157.725005,9.989337e-09,2020-09-16 19:28:07
3,143.544388,18.529515,36.941449,5.157298e-11,2020-09-16 20:15:33
4,14203.219302,1833.429928,3663.026267,5.102965e-09,2020-09-17 00:21:13
5,576.034022,74.356343,147.592964,2.069570e-10,2020-09-18 05:26:54
6,3846.609344,496.516072,987.800659,1.381983e-09,2020-09-20 13:52:07
7,291.543450,37.625893,75.102954,1.047350e-10,2020-10-06 12:53:08
8,25.564644,3.280733,6.558273,9.157992e-12,2020-12-06 09:20:37
9,78.414536,9.940579,19.897154,2.791746e-11,2021-05-26 00:50:16


### Writing swaps, mints, burns, reserves

In [88]:
hkmt_usdt_df.to_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_swaps.pkl")
hkmt_usdt_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_reserves.pkl")
hkmt_usdt_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_mints.pkl")
hkmt_usdt_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_burns.pkl")

### Loading swaps, mints, burns, reserves

In [89]:
hkmt_usdt_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_swaps.pkl")
hkmt_usdt_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_reserves.pkl")
hkmt_usdt_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_mints.pkl")
hkmt_usdt_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_burns.pkl")

# FEI/ETH pool

## Getting FEI/ETH swaps, mints, burns, reserves

In [ ]:
fei_weth_history = get_pool_v2_history(contract_id='0x94b0a3d511b6ecdb17ebf877278ab030acb0a878', range_limit=50)
fei_weth_df = pool_history_to_df(fei_weth_history)
fei_weth_df

In [95]:
fei_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x94b0a3d511b6ecdb17ebf877278ab030acb0a878')
fei_weth_reserves_df = pool_reserves_to_df(fei_weth_reserves_history)
fei_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.72it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,1.337230e+09,630522.677036,2.537871e+09,5.830436e+07,27452.763852,2021-04-03
1,1.341220e+09,628533.678068,2.604474e+09,6.647175e+07,31306.853434,2021-04-04
2,1.337334e+09,610770.369281,2.561514e+09,8.717716e+07,40829.752667,2021-04-05
3,1.323221e+09,598300.386209,2.531489e+09,5.256349e+07,24003.259833,2021-04-06
4,1.192320e+09,490330.663557,1.925378e+09,2.657121e+08,122248.657720,2021-04-07
...,...,...,...,...,...,...
217,1.501845e+08,33358.812751,3.008810e+08,9.439718e+06,2129.006495,2021-11-06
218,1.520115e+08,32961.427400,3.038065e+08,5.412572e+06,1185.020670,2021-11-07
219,1.548270e+08,32363.289968,3.102908e+08,3.921755e+06,831.158745,2021-11-08
220,1.541618e+08,32510.246132,3.081654e+08,1.157095e+07,2429.260301,2021-11-09


In [96]:
fei_weth_mints_history = get_pool_v2_mints(contract_id='0x94b0a3d511b6ecdb17ebf877278ab030acb0a878', range_limit=2)
fei_weth_mints_df = pool_mints_to_df(fei_weth_mints_history)
fei_weth_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.77it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1.315771e+09,639235.592414,1.318727e+09,2.900151e+07,2021-04-03 18:59:46
1,7.688361e+03,3.994820,1.578041e+04,1.666953e+02,2021-04-03 19:10:31
2,1.255904e+05,64.904372,2.567463e+05,2.716398e+03,2021-04-03 19:19:25
3,2.073553e+01,0.009700,4.033823e+01,4.484614e-01,2021-04-03 19:19:42
4,6.508717e+02,0.333262,1.327281e+03,1.408814e+01,2021-04-03 19:23:25
...,...,...,...,...,...
594,1.333098e+08,36805.045562,2.674915e+08,2.136795e+06,2021-10-08 10:13:06
595,1.309333e+08,37275.415575,2.618830e+08,2.130943e+06,2021-10-10 01:33:20
596,1.324539e+08,37577.176787,2.643709e+08,2.151832e+06,2021-10-10 19:04:42
597,4.278637e+04,12.138499,8.539863e+04,6.951029e+02,2021-10-10 19:04:56


In [97]:
fei_weth_burns_history = get_pool_v2_burns(contract_id='0x94b0a3d511b6ecdb17ebf877278ab030acb0a878', range_limit=2)
fei_weth_burns_df = pool_burns_to_df(fei_weth_burns_history)
fei_weth_burns_df

100%|██████████| 2/2 [00:01<00:00,  1.25it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,6.517701e+02,0.304577,1.268090e+03,1.408814e+01,2021-04-03 19:56:32
1,2.201983e+04,10.277614,4.288282e+04,4.756753e+02,2021-04-03 20:00:59
2,3.826946e+03,1.778466,7.337225e+03,8.249027e+01,2021-04-03 20:49:31
3,4.349987e+03,2.034685,8.270638e+03,9.406615e+01,2021-04-03 22:28:41
4,3.641036e-02,0.000017,6.908305e-02,7.873546e-04,2021-04-03 22:29:18
...,...,...,...,...,...
520,1.316554e+08,37276.459551,2.618904e+08,2.136850e+06,2021-10-10 01:33:20
521,1.324967e+08,37589.319208,2.639461e+08,2.152527e+06,2021-10-10 18:48:10
522,1.039316e+03,0.296067,2.083135e+03,1.691675e+01,2021-10-11 22:19:02
523,4.287733e+02,0.094292,8.565958e+02,6.116372e+00,2021-11-05 08:06:51


### Writing swaps, mints, burns, reserves to pickle files

In [98]:
fei_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_swaps.pkl")
fei_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_reserves.pkl")
fei_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_mints.pkl")
fei_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [99]:
fei_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_swaps.pkl")
fei_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_reserves.pkl")
fei_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_mints.pkl")
fei_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_burns.pkl")

# ETH/USDC pool

In [ ]:
weth_usdc_history = get_pool_v2_history(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=2928)
weth_usdc_df = pool_history_to_df(weth_usdc_history)
weth_usdc_df

In [108]:
weth_usdc_reserves_history = get_pool_v2_reserves_history(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc')
weth_usdc_reserves_df = pool_reserves_to_df(weth_usdc_reserves_history)
weth_usdc_reserves_df

100%|██████████| 2/2 [00:06<00:00,  3.05s/it]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,9.890000e-01,0.004909,9.890000e-01,1.100000e-02,0.000054,2020-05-05
1,9.886890e-01,0.004910,9.886890e-01,1.689000e-03,0.000008,2020-05-06
2,9.886900e-01,0.004910,9.886900e-01,0.000000e+00,0.000000,2020-05-07
3,6.574471e+00,0.030998,6.574471e+00,0.000000e+00,0.000000,2020-05-08
4,6.562244e+00,0.031061,6.562244e+00,3.975130e-01,0.001937,2020-05-11
...,...,...,...,...,...,...
548,1.123080e+08,24896.942631,2.244618e+08,3.790653e+07,8532.386196,2021-11-06
549,8.736188e+07,18999.899193,1.749496e+08,3.393141e+07,7411.069188,2021-11-07
550,8.961766e+07,18660.641566,1.793024e+08,5.005029e+07,10569.022766,2021-11-08
551,1.013231e+08,21412.951564,2.028954e+08,4.742757e+07,9909.756985,2021-11-09


In [115]:
weth_usdc_mints_history = get_pool_v2_mints(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=34)
weth_usdc_mints_df = pool_mints_to_df(weth_usdc_mints_history)
weth_usdc_mints_df

100%|██████████| 34/34 [00:38<00:00,  1.12s/it]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1.000000e+00,0.004854,1.000000e+00,6.967330e-08,2020-05-05 21:07:12
1,3.263000e-03,0.000016,3.263000e-03,2.299449e-10,2020-05-07 14:56:15
2,1.419000e-02,0.000070,1.419000e-02,9.999739e-10,2020-05-08 02:29:14
3,5.000000e-02,0.000235,4.741450e-02,3.332596e-09,2020-05-08 20:18:28
4,5.357800e-01,0.002500,5.148727e-01,3.547452e-08,2020-05-08 21:50:40
...,...,...,...,...,...
32861,4.738854e+02,0.099850,9.473044e+02,3.851792e-06,2021-11-10 02:48:12
32862,3.846112e+07,8139.103496,7.694166e+07,3.132628e-01,2021-11-10 07:41:21
32863,4.707125e+02,0.100000,9.421652e+02,3.841242e-06,2021-11-10 10:15:20
32864,3.921999e+05,82.977689,7.846313e+05,3.193886e-03,2021-11-10 11:11:45


In [117]:
weth_usdc_burns_history = get_pool_v2_burns(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=24)
weth_usdc_burns_df = pool_burns_to_df(weth_usdc_burns_history)
weth_usdc_burns_df

100%|██████████| 24/24 [00:27<00:00,  1.15s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,3.262000e-03,0.000016,3.263003e-03,2.299449e-10,2020-05-07 15:47:25
1,1.418900e-02,0.000070,1.419001e-02,9.999739e-10,2020-05-08 02:43:59
2,7.812840e-01,0.006539,7.812847e-01,6.960485e-08,2020-05-12 20:18:50
3,9.953090e-01,0.004114,8.361725e-01,6.217433e-08,2020-05-13 23:48:21
4,1.097105e+00,0.005132,1.047697e+00,7.289922e-08,2020-05-14 14:24:41
...,...,...,...,...,...
23307,2.613006e+03,0.543883,5.227160e+03,2.112073e-05,2021-11-09 10:28:54
23308,3.257345e+05,69.085029,6.515458e+05,2.656415e-03,2021-11-10 01:06:40
23309,1.090435e+06,229.948711,2.180152e+06,8.866823e-03,2021-11-10 02:42:27
23310,1.573420e+02,0.033234,3.146902e+02,1.280273e-06,2021-11-10 10:59:39


### Writing swaps, mints, burns, reserves into pickle files

In [118]:
weth_usdc_df.to_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_swaps.pkl")
weth_usdc_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_reserves.pkl")
weth_usdc_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_mints.pkl")
weth_usdc_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [119]:
weth_usdc_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_swaps.pkl")
weth_usdc_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_reserves.pkl")
weth_usdc_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_mints.pkl")
weth_usdc_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_burns.pkl")

# AXS/ETH v2 pool - NFT

In [6]:
axs_weth_v2_history = get_pool_v2_history(contract_id='0xa040832b1e5026da6dc843e6773a9e4d02fff9b1', range_limit=5)
axs_weth_v2_df = pool_history_to_df(axs_weth_v2_history)
axs_weth_v2_df

100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,WETH,AXS,1.000000,271.895117,3265.028270,2021-05-03 01:42:22
1,WETH,AXS,0.400000,87.762331,1238.364115,2021-05-03 01:42:28
2,WETH,AXS,0.190000,0.048255,572.499245,2021-05-03 01:42:48
3,WETH,AXS,0.010000,0.002537,30.131539,2021-05-03 01:42:48
4,WETH,AXS,0.300000,59.387987,921.135675,2021-05-03 01:42:48
...,...,...,...,...,...,...
4458,AXS,WETH,6.403259,0.182930,819.453147,2021-11-11 01:34:40
4459,WETH,AXS,0.029738,1.089641,137.462794,2021-11-11 01:51:29
4460,WETH,AXS,0.198250,6.730455,946.816312,2021-11-11 04:26:15
4461,AXS,WETH,9.410000,0.268806,1202.643879,2021-11-11 08:44:36


In [8]:
axs_weth_v2_reserves_history = get_pool_v2_reserves_history(contract_id='0xa040832b1e5026da6dc843e6773a9e4d02fff9b1')
axs_weth_v2_reserves_df = pool_reserves_to_df(axs_weth_v2_reserves_history)
axs_weth_v2_reserves_df

100%|██████████| 2/2 [00:00<00:00,  3.25it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,2235.318021,6.589013,43248.099360,5631.549029,412.261553,2021-05-03
1,2468.971504,5.971757,39933.495620,876.346517,2.191252,2021-05-04
2,2407.284859,6.133823,42093.892594,1233.886645,3.094878,2021-05-05
3,2603.981326,5.557229,38883.103492,1681.480184,4.037121,2021-05-06
4,2148.038360,5.171886,35978.200367,2004.968493,4.749660,2021-05-07
...,...,...,...,...,...,...
171,93.720378,3.246412,29904.748186,13.319140,0.446260,2021-11-07
172,97.091996,3.137604,29966.847564,41.115158,1.406405,2021-11-08
173,98.147476,3.107572,29688.149794,45.542305,1.272922,2021-11-09
174,100.055361,3.049939,28068.271846,18.431023,0.554382,2021-11-10


In [9]:
axs_weth_v2_mints_history = get_pool_v2_mints(contract_id='0xa040832b1e5026da6dc843e6773a9e4d02fff9b1', range_limit=2)
axs_weth_v2_mints_df = pool_mints_to_df(axs_weth_v2_mints_history)
axs_weth_v2_mints_df

100%|██████████| 2/2 [00:00<00:00,  4.35it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1866.449528,5.847000,17615.215682,1.044659e+02,2021-05-03 01:42:22
1,25.000000,0.071740,474.404409,1.332144e+00,2021-05-03 18:14:10
2,280.000000,0.803485,5298.366107,1.492001e+01,2021-05-03 18:31:59
3,50.000000,0.096979,380.827232,2.155080e+00,2021-05-15 09:01:25
4,3.982368,0.010916,41.793753,2.038269e-01,2021-05-16 02:37:43
...,...,...,...,...,...
60,16.962663,0.408726,2635.759821,4.554845e-07,2021-08-28 15:14:19
61,35.673170,0.855174,6629.921820,9.511581e-07,2021-09-05 07:21:31
62,28.000000,1.000116,6766.518537,8.891942e-07,2021-10-06 08:58:39
63,4.352830,0.137023,1112.618896,1.285026e-07,2021-10-20 15:56:57


In [10]:
axs_weth_v2_burns_history = get_pool_v2_burns(contract_id='0xa040832b1e5026da6dc843e6773a9e4d02fff9b1', range_limit=2)
axs_weth_v2_burns_df = pool_burns_to_df(axs_weth_v2_burns_history)
axs_weth_v2_burns_df

100%|██████████| 2/2 [00:00<00:00,  4.44it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,26.456386,0.067974,468.479353,1.332144e+00,2021-05-06 05:26:36
1,326.295177,0.692936,4782.409399,1.492001e+01,2021-05-07 07:54:27
2,2423.697774,4.615325,34391.396160,1.044659e+02,2021-05-13 11:20:29
3,48.409812,0.103813,348.682796,2.175059e+00,2021-05-18 00:31:52
4,151.944075,0.271016,1455.393578,5.617203e+00,2021-06-02 10:25:06
5,608.541769,1.030020,5809.629691,2.183182e+01,2021-06-03 06:40:18
6,797.524914,1.410775,8139.684065,2.923736e+01,2021-06-03 10:27:19
7,440.578025,0.781316,4386.521501,1.612396e+01,2021-06-03 20:55:08
8,461.742241,0.748523,3927.824987,1.596272e+01,2021-06-05 20:19:51
9,94.302815,0.151797,751.189716,3.240915e+00,2021-06-09 00:28:03


### Writing swaps, mints, burns, reserves into pickle files

In [11]:
axs_weth_v2_df.to_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_swaps.pkl")
axs_weth_v2_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_reserves.pkl")
axs_weth_v2_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_mints.pkl")
axs_weth_v2_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [12]:
axs_weth_v2_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_swaps.pkl")
axs_weth_v2_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_reserves.pkl")
axs_weth_v2_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_mints.pkl")
axs_weth_v2_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_burns.pkl")

# MANA/ETH pool (Decentraland/Ethereum) - NFT

In [ ]:
mana_weth_history = get_pool_v2_history(contract_id='0x11b1f53204d03e5529f09eb3091939e4fd8c9cf3', range_limit=72)
mana_weth_df = pool_history_to_df(mana_weth_history)
mana_weth_df

In [18]:
mana_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x11b1f53204d03e5529f09eb3091939e4fd8c9cf3')
mana_weth_reserves_df = pool_reserves_to_df(mana_weth_reserves_history)
mana_weth_reserves_df

100%|██████████| 2/2 [00:01<00:00,  1.65it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,1.313154e+04,2.455654,5.260017e+02,2.759552e+02,0.050000,2020-05-18
1,1.995563e+04,3.402474,1.456969e+03,1.545187e+03,0.268593,2020-05-19
2,2.320825e+04,4.142890,1.747858e+03,5.835603e+03,0.971594,2020-05-20
3,2.287628e+04,4.203910,1.674104e+03,1.630945e+03,0.304859,2020-05-21
4,4.713547e+04,9.083270,3.766017e+03,1.150966e+04,2.419877,2020-05-22
...,...,...,...,...,...,...
538,1.406560e+06,858.125110,7.901340e+06,3.750751e+05,231.599542,2021-11-07
539,1.470613e+06,821.824572,7.896708e+06,6.330063e+05,365.717445,2021-11-08
540,1.503168e+06,806.132295,7.645894e+06,1.358138e+06,718.900592,2021-11-09
541,1.503989e+06,808.678993,7.479941e+06,1.843973e+06,1008.822995,2021-11-10


In [17]:
mana_weth_mints_history = get_pool_v2_mints(contract_id='0x11b1f53204d03e5529f09eb3091939e4fd8c9cf3', range_limit=2)
mana_weth_mints_df = pool_mints_to_df(mana_weth_mints_history)
mana_weth_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.29it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,8138.778341,1.420382,301.443838,107.518245,2020-05-18 19:40:36
1,197.637271,0.036959,7.921571,2.702542,2020-05-18 22:50:04
2,5071.078019,0.948313,203.128934,69.343211,2020-05-18 23:25:52
3,2238.013091,0.371897,159.418133,28.845815,2020-05-19 00:35:15
4,2810.821822,0.482834,202.898546,36.833688,2020-05-19 06:34:29
...,...,...,...,...,...
934,116.845729,0.099855,860.886069,2.157171,2021-10-31 06:45:25
935,497.691396,0.309301,2786.946639,7.670562,2021-11-06 21:42:55
936,324.989280,0.199857,1820.601978,4.981986,2021-11-07 02:16:58
937,1529.721602,0.911172,8565.212954,23.017443,2021-11-10 04:53:04


In [16]:
mana_weth_burns_history = get_pool_v2_burns(contract_id='0x11b1f53204d03e5529f09eb3091939e4fd8c9cf3', range_limit=2)
mana_weth_burns_df = pool_burns_to_df(mana_weth_burns_history)
mana_weth_burns_df

100%|██████████| 2/2 [00:01<00:00,  1.71it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1119.607426,0.205747,81.280926,15.167027,2020-05-22 04:17:12
1,460.941138,0.085813,35.486995,6.281675,2020-05-23 19:44:19
2,2774.958609,0.466815,222.206766,35.891576,2020-05-31 05:19:22
3,24058.809930,4.143955,1999.222922,314.514600,2020-06-05 18:52:05
4,3003.841988,0.507565,240.105825,38.878333,2020-06-07 19:04:19
...,...,...,...,...,...
669,1895.427585,1.029468,9214.903318,27.347885,2021-11-05 15:40:42
670,166.364936,0.109201,981.413788,2.637382,2021-11-06 01:49:48
671,204.600245,0.135663,1214.871122,3.258817,2021-11-06 09:42:49
672,328.730402,0.197591,1801.002805,4.981986,2021-11-07 05:25:03


### Writing swaps, mints, burns, reserves into pickle files

In [29]:
mana_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_swaps.pkl")
mana_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_reserves.pkl")
mana_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_mints.pkl")
mana_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [20]:
mana_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_swaps.pkl")
mana_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_reserves.pkl")
mana_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_mints.pkl")
mana_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_burns.pkl")

# ETH/ENJ pool - NFT

In [22]:
enj_eth_history = get_pool_v2_history(contract_id='0xe56c60b5f9f7b5fc70de0eb79c6ee7d00efa2625', range_limit=76)
enj_eth_df = pool_history_to_df(enj_eth_history)
enj_eth_df

 95%|█████████▌| 76/80 [01:46<00:03,  1.24it/s]

list index out of range
list index out of range


 96%|█████████▋| 77/80 [01:46<00:01,  1.60it/s]

list index out of range


 99%|█████████▉| 79/80 [01:47<00:00,  2.37it/s]

list index out of range
list index out of range


100%|██████████| 80/80 [01:47<00:00,  1.35s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,WETH,ENJ,0.250000,329.243700,53.924783,2020-05-19 00:27:03
1,WETH,ENJ,0.003249,4.206203,0.690529,2020-05-19 01:25:30
2,WETH,ENJ,0.050000,64.528525,10.633417,2020-05-19 01:48:49
3,WETH,ENJ,0.050000,64.156216,10.632978,2020-05-19 01:52:46
4,WETH,ENJ,0.010000,12.786805,2.124114,2020-05-19 01:56:57
...,...,...,...,...,...,...
73719,ENJ,WETH,297.982539,0.192195,894.570731,2021-11-11 06:15:26
73720,WETH,ENJ,0.178425,274.998641,836.931593,2021-11-11 08:03:50
73721,ENJ,WETH,1345.341905,0.864462,4049.598348,2021-11-11 08:23:45
73722,WETH,ENJ,0.214100,332.367906,1008.784735,2021-11-11 09:25:56


In [23]:
enj_eth_reserves_history = get_pool_v2_reserves_history(contract_id='0xe56c60b5f9f7b5fc70de0eb79c6ee7d00efa2625')
enj_eth_reserves_df = pool_reserves_to_df(enj_eth_reserves_history)
enj_eth_reserves_df

100%|██████████| 2/2 [00:01<00:00,  1.69it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,14.596187,19609.896031,6.256915e+03,0.000000,0.000000,2020-05-18
1,35.608058,45643.923958,1.503812e+04,4.334663,5636.075761,2020-05-19
2,44.083044,55306.165361,1.852330e+04,6.154577,8060.401355,2020-05-20
3,68.513132,76046.446474,2.728367e+04,39.970996,43953.282808,2020-05-21
4,70.230793,82183.521342,2.917812e+04,15.703982,18107.270743,2020-05-22
...,...,...,...,...,...,...
538,197.158411,274486.674468,1.816373e+06,84.303046,113268.199448,2021-11-07
539,189.551810,285956.100674,1.816394e+06,104.945458,150403.609035,2021-11-08
540,183.841111,291757.647168,1.744292e+06,27.593816,43452.417673,2021-11-09
541,181.090315,296421.520097,1.678366e+06,48.458147,76004.100494,2021-11-10


In [24]:
enj_eth_mints_history = get_pool_v2_mints(contract_id='0xe56c60b5f9f7b5fc70de0eb79c6ee7d00efa2625', range_limit=2)
enj_eth_mints_df = pool_mints_to_df(enj_eth_mints_history)
enj_eth_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.39it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,12.826616,17232.487792,2738.593201,470.143074,2020-05-18 21:58:29
1,1.569474,2108.580074,672.783099,57.527058,2020-05-18 22:48:38
2,0.200096,268.828166,85.774806,7.334269,2020-05-18 22:49:38
3,2.356681,3060.607620,1002.815881,84.926512,2020-05-19 01:17:18
4,0.240934,315.332743,101.882671,8.715742,2020-05-19 13:20:41
...,...,...,...,...,...
943,0.400000,869.485563,2309.997651,12.666224,2021-09-24 09:47:10
944,0.115844,270.195427,678.567387,3.795850,2021-09-29 11:57:25
945,0.819787,1219.152027,7340.724844,20.955602,2021-11-02 14:02:25
946,0.219957,347.839101,2011.400400,5.785439,2021-11-03 12:03:43


In [25]:
enj_eth_burns_history = get_pool_v2_burns(contract_id='0xe56c60b5f9f7b5fc70de0eb79c6ee7d00efa2625', range_limit=2)
enj_eth_burns_df = pool_burns_to_df(enj_eth_burns_history)
enj_eth_burns_df

100%|██████████| 2/2 [00:01<00:00,  1.96it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.324013,359.640139,128.844953,10.779454,2020-05-22 02:34:24
1,0.297889,354.773011,124.207311,10.262332,2020-05-22 18:42:33
2,4.020403,4879.430957,1685.933227,139.747002,2020-05-24 09:55:27
3,3.921471,4759.360616,1644.446714,136.308185,2020-05-24 09:56:27
4,11.301433,13591.069990,4615.327351,390.992199,2020-05-24 18:34:17
...,...,...,...,...,...
730,0.070855,133.337889,572.808193,2.065935,2021-10-28 08:19:44
731,0.256608,383.696403,2203.257552,6.656577,2021-10-30 15:19:10
732,0.908724,1325.251976,7793.351212,23.254600,2021-10-30 23:03:31
733,0.432006,594.039290,3977.594784,10.514424,2021-11-07 18:52:45


### Writing swaps, mints, burns, reserves into pickle files

In [28]:
enj_eth_df.to_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_swaps.pkl")
enj_eth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_reserves.pkl")
enj_eth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_mints.pkl")
enj_eth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [30]:
enj_eth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_swaps.pkl")
enj_eth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_reserves.pkl")
enj_eth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_mints.pkl")
enj_eth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_burns.pkl")

# SAND/ETH pool - NFT

In [38]:
sand_weth_history = get_pool_v2_history(contract_id='0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74', range_limit=130)
sand_weth_df = pool_history_to_df(sand_weth_history)
sand_weth_df

100%|██████████| 130/130 [03:19<00:00,  1.53s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,WETH,SAND,0.017650,126.469664,3.647599,2020-08-20 15:46:06
1,WETH,SAND,0.400000,2413.756323,178.145088,2020-08-20 15:56:12
2,WETH,SAND,0.476309,2155.508531,212.249884,2020-08-20 15:57:40
3,WETH,SAND,0.031877,123.596424,13.288490,2020-08-20 16:02:32
4,WETH,SAND,0.008500,32.612732,3.539984,2020-08-20 16:20:56
...,...,...,...,...,...,...
128512,WETH,SAND,0.630000,1257.292744,2981.313401,2021-11-11 10:18:42
128513,WETH,SAND,0.100000,199.535403,473.188066,2021-11-11 10:20:12
128514,WETH,SAND,2.348163,4682.679239,11098.850877,2021-11-11 10:26:28
128515,SAND,WETH,11649.064883,5.801709,27303.030454,2021-11-11 10:40:19


In [32]:
sand_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74')
sand_weth_reserves_df = pool_reserves_to_df(sand_weth_reserves_history)
sand_weth_reserves_df

100%|██████████| 2/2 [00:02<00:00,  1.35s/it]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,4.620900e+04,6.473900,5.360654e+03,1.362405e+05,24.472452,2020-08-20
1,8.011954e+04,9.442597,7.270852e+03,1.862165e+05,26.147471,2020-08-21
2,8.079633e+04,10.412832,8.245477e+03,7.209562e+04,9.277340,2020-08-22
3,1.145013e+05,14.152003,1.110878e+04,5.906999e+04,7.473852,2020-08-23
4,1.199340e+05,14.999432,1.218586e+04,1.398868e+05,20.077351,2020-08-24
...,...,...,...,...,...,...
444,7.709895e+06,4469.703849,4.115774e+07,1.839318e+06,1066.257574,2021-11-07
445,7.988303e+06,4286.073315,4.118310e+07,3.775639e+06,2023.569226,2021-11-08
446,8.304657e+06,4158.405242,3.939862e+07,3.644201e+06,1870.775753,2021-11-09
447,8.435944e+06,4150.132166,3.836007e+07,4.702694e+06,2445.618720,2021-11-10


In [33]:
sand_weth_mints_history = get_pool_v2_mints(contract_id='0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74', range_limit=2)
sand_weth_mints_df = pool_mints_to_df(sand_weth_mints_history)
sand_weth_mints_df

100%|██████████| 2/2 [00:02<00:00,  1.08s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,1250.000000,0.168750,69.715655,14.523688,2020-08-20 15:26:19
1,3000.000000,0.405000,166.878223,34.856850,2020-08-20 15:29:38
2,14171.000000,2.032416,842.268420,169.702062,2020-08-20 15:49:30
3,1250.000000,0.318767,265.218003,19.952492,2020-08-20 16:02:23
4,10000.000000,2.470893,2058.665607,157.087106,2020-08-20 16:42:48
...,...,...,...,...,...
1995,6501.878843,0.498644,591.403567,53.076827,2020-12-01 17:38:53
1996,119.122908,0.009126,10.869397,0.971931,2020-12-01 19:45:46
1997,1419.066263,0.107953,129.199030,11.537341,2020-12-01 20:11:12
1998,2899.167241,0.220488,263.377507,23.567692,2020-12-01 20:15:13


In [34]:
sand_weth_burns_history = get_pool_v2_burns(contract_id='0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74', range_limit=2)
sand_weth_burns_df = pool_burns_to_df(sand_weth_burns_history)
sand_weth_burns_df

100%|██████████| 2/2 [00:02<00:00,  1.19s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,14443.767380,1.752435,1409.746715,157.087106,2020-08-24 17:33:44
1,18821.089713,2.694975,2151.411546,221.784224,2020-08-25 03:49:23
2,14121.242828,3.530601,2685.764707,219.172492,2020-08-27 15:22:02
3,17412.625149,3.007004,2404.186361,223.913860,2020-08-29 17:24:15
4,13464.195804,2.271845,1826.248541,171.121556,2020-08-30 06:57:30
...,...,...,...,...,...
1995,4897.017872,1.199387,8273.019079,60.000000,2021-09-09 23:04:57
1996,2075.032593,0.498905,3490.012333,25.184565,2021-09-10 02:36:35
1997,4589.506444,1.116136,7673.015477,56.018826,2021-09-10 06:33:41
1998,3634.904700,0.868581,5863.513224,43.976966,2021-09-10 08:57:51


### Writing swaps, mints, burns, reserves into pickle files

In [41]:
sand_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_swaps.pkl")
sand_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_reserves.pkl")
sand_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_mints.pkl")
sand_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [42]:
sand_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_swaps.pkl")
sand_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_reserves.pkl")
sand_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_mints.pkl")
sand_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_burns.pkl")

# ALICE/ETH pool - NFT

In [44]:
alice_weth_history = get_pool_v2_history(contract_id='0x30bc873193466dc59fc08159460c49f26f609795', range_limit=10)
alice_weth_df = pool_history_to_df(alice_weth_history)
alice_weth_df

 90%|█████████ | 9/10 [00:10<00:00,  1.13it/s]

list index out of range


100%|██████████| 10/10 [00:10<00:00,  1.07s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,ALICE,WETH,12.839028,0.174107,284.992234,2021-03-16 08:37:32
1,ALICE,WETH,2.000000,0.021126,36.733660,2021-03-16 12:04:44
2,ALICE,WETH,14.224945,0.120341,224.296642,2021-03-17 07:37:35
3,WETH,ALICE,0.187000,20.060961,329.448128,2021-03-17 11:36:13
4,ALICE,WETH,10.000000,0.105862,169.483118,2021-03-17 12:26:52
...,...,...,...,...,...,...
7598,ALICE,WETH,705.208811,2.367885,11208.473277,2021-11-11 10:10:10
7599,ALICE,WETH,448.428103,1.439385,6813.384682,2021-11-11 10:11:13
7600,ALICE,WETH,517.725320,1.601747,7582.250412,2021-11-11 10:11:46
7601,ALICE,WETH,1166.989120,3.392493,16054.065853,2021-11-11 10:12:47


In [47]:
alice_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x30bc873193466dc59fc08159460c49f26f609795')
alice_weth_reserves_df = pool_reserves_to_df(alice_weth_reserves_history)
alice_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  3.11it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,6.000000,0.101598,367.355029,0.000000,0.000000,2021-03-15
1,67.100212,0.689707,2430.779070,14.839028,0.195232,2021-03-16
2,68.634374,0.675505,2232.418760,46.915728,0.438203,2021-03-17
3,74.634374,0.621349,2098.103332,6.000000,0.054155,2021-03-22
4,85.550512,0.544140,1817.052880,106.134748,1.034504,2021-03-23
...,...,...,...,...,...,...
232,24564.998908,87.756435,809922.648413,10989.471179,38.542690,2021-11-07
233,25269.449095,85.443579,814698.938656,13069.776295,45.963698,2021-11-08
234,25843.637657,83.655710,798216.894229,12234.169168,37.421361,2021-11-09
235,27501.138208,78.727239,726965.361910,13115.328077,39.128551,2021-11-10


In [48]:
alice_weth_mints_history = get_pool_v2_mints(contract_id='0x30bc873193466dc59fc08159460c49f26f609795', range_limit=2)
alice_weth_mints_df = pool_mints_to_df(alice_weth_mints_history)
alice_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  4.28it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,6.000000,0.101598,367.355029,7.807616e-07,2021-03-15 10:52:49
1,46.261184,0.783341,2324.406045,6.019826e-06,2021-03-16 08:37:29
2,16737.068671,96.832368,339371.842500,1.267791e-03,2021-03-25 16:19:41
3,993.683353,7.264836,23374.408643,8.459013e-05,2021-03-25 19:34:30
4,2851.000000,20.025448,64780.308064,2.377853e-04,2021-03-26 02:52:05
5,7118.442317,50.000000,161393.357047,5.937079e-04,2021-03-26 02:54:47
6,4271.065390,30.000000,96854.620482,3.562247e-04,2021-03-26 02:56:50
7,100.000000,0.786183,2556.857781,8.822277e-06,2021-03-26 12:31:12
8,1075.901600,8.270369,27162.727663,9.384072e-05,2021-03-26 15:23:33
9,301.370000,2.437962,8134.923425,2.696432e-05,2021-03-26 20:41:38


In [49]:
alice_weth_burns_history = get_pool_v2_burns(contract_id='0x30bc873193466dc59fc08159460c49f26f609795', range_limit=2)
alice_weth_burns_df = pool_burns_to_df(alice_weth_burns_history)
alice_weth_burns_df

100%|██████████| 2/2 [00:00<00:00,  4.21it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,970.524283,7.451131,25321.238237,8.459013e-05,2021-03-27 04:44:20
1,44.370849,0.425158,1447.792493,4.315723e-06,2021-03-28 17:30:13
2,20.643398,0.192727,650.151355,1.981515e-06,2021-03-29 05:49:22
3,155.562830,1.505671,5140.720429,1.520242e-05,2021-03-29 08:42:17
4,57.815938,0.433081,1726.990931,4.965505e-06,2021-04-02 13:23:20
5,119.999999,0.832313,3487.019509,9.911754e-06,2021-04-03 15:06:29
6,6197.517242,37.706955,162681.978334,4.780017e-04,2021-04-10 11:04:23
7,16679.110145,98.610485,423748.268984,1.267791e-03,2021-04-11 02:51:04
8,1194.704911,7.557609,32169.567383,9.384072e-05,2021-04-12 15:23:30
9,192.252928,0.954394,4735.915000,1.336820e-05,2021-04-16 04:28:06


### Writing swaps, mints, burns, reserves into pickle files

In [50]:
alice_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_swaps.pkl")
alice_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_reserves.pkl")
alice_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_mints.pkl")
alice_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [51]:
alice_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_swaps.pkl")
alice_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_reserves.pkl")
alice_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_mints.pkl")
alice_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_burns.pkl")

# DOGE/ETH pool

In [55]:
doge_weth_history = get_pool_v2_history(contract_id='0xc0067d751fb1172dbab1fa003efe214ee8f419b6', range_limit=15)
doge_weth_df = pool_history_to_df(doge_weth_history)
doge_weth_df

100%|██████████| 15/15 [00:18<00:00,  1.47it/s]

list index out of range
list index out of range


100%|██████████| 15/15 [00:18<00:00,  1.21s/it]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,WETH,DOGE,1.000000,131189.389055,1237.901160,2021-01-18 21:14:49
1,WETH,DOGE,0.007725,1000.000000,9.508499,2021-01-18 21:18:57
2,WETH,DOGE,0.041338,5347.743915,50.895183,2021-01-18 21:29:35
3,WETH,DOGE,0.025787,3333.000000,31.799822,2021-01-18 21:31:54
4,WETH,DOGE,0.300000,38736.888276,372.869812,2021-01-18 23:33:09
...,...,...,...,...,...,...
12114,DOGE,WETH,33842.896541,1.860260,8559.376795,2021-11-11 00:51:19
12115,WETH,DOGE,3.066334,55343.622098,14193.795894,2021-11-11 04:30:56
12116,DOGE,WETH,55343.618509,3.103351,14365.145218,2021-11-11 04:30:56
12117,WETH,DOGE,5.618417,100000.000000,26007.173571,2021-11-11 04:30:56


In [54]:
doge_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0xc0067d751fb1172dbab1fa003efe214ee8f419b6')
doge_weth_reserves_df = pool_reserves_to_df(doge_weth_reserves_history)
doge_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.50it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,5.815876e+07,449.362514,1.117025e+06,1.796070e+05,1.374850,2021-01-18
1,1.053052e+08,690.380932,1.924844e+06,8.014429e+06,56.096709,2021-01-19
2,1.048894e+08,693.127828,1.756477e+06,4.933253e+05,3.253104,2021-01-22
3,1.068894e+08,680.197323,1.847699e+06,2.020000e+06,13.058171,2021-01-25
4,1.083894e+08,670.812012,1.807903e+06,1.500000e+06,9.385311,2021-01-26
...,...,...,...,...,...,...
289,1.092428e+07,632.966905,5.827256e+06,2.376571e+05,13.770956,2021-11-07
290,1.088065e+07,635.657490,6.075537e+06,8.608951e+05,51.732961,2021-11-08
291,1.095118e+07,631.621488,6.072999e+06,3.327703e+05,19.589827,2021-11-09
292,1.118467e+07,618.504353,5.635877e+06,4.091358e+05,23.099808,2021-11-10


In [56]:
doge_weth_mints_history = get_pool_v2_mints(contract_id='0xc0067d751fb1172dbab1fa003efe214ee8f419b6', range_limit=2)
doge_weth_mints_df = pool_mints_to_df(doge_weth_mints_history)
doge_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  4.64it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1.000000e+07,75.000000,9.236720e+04,0.273861,2021-01-18 21:14:49
1,1.604497e+07,123.805938,3.059632e+05,0.445688,2021-01-18 22:41:33
2,2.050000e+07,158.181727,3.911640e+05,0.569437,2021-01-18 22:49:49
3,1.179340e+07,91.000000,2.254626e+05,0.327590,2021-01-18 23:07:24
4,2.868813e+06,20.000000,5.296842e+04,0.075734,2021-01-19 02:19:07
5,2.135449e+06,14.000000,3.986648e+04,0.054666,2021-01-19 17:18:20
6,3.706187e+07,242.977582,6.774434e+05,0.948755,2021-01-19 18:23:45
7,5.307253e+03,0.050000,1.300749e+02,0.000163,2021-01-28 11:24:21
8,1.121247e+05,1.998984,5.392716e+03,0.004723,2021-01-28 23:07:30
9,2.298049e+03,0.119647,3.125922e+02,0.000165,2021-01-29 06:34:28


In [57]:
doge_weth_burns_history = get_pool_v2_burns(contract_id='0xc0067d751fb1172dbab1fa003efe214ee8f419b6', range_limit=2)
doge_weth_burns_df = pool_burns_to_df(doge_weth_burns_history)
doge_weth_burns_df

100%|██████████| 2/2 [00:00<00:00,  4.74it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,6.894842e+04,2.655201,7.543399e+03,0.004251,2021-01-29 14:15:04
1,3.185133e+03,0.084567,2.276126e+02,0.000163,2021-01-30 05:01:23
2,2.528739e+03,0.112135,3.666145e+02,0.000165,2021-02-08 06:45:07
3,3.643306e+03,0.169095,5.840740e+02,0.000243,2021-02-09 02:51:38
4,1.480797e+04,0.414450,1.655572e+03,0.000765,2021-02-20 18:36:01
5,3.023294e+03,0.096621,3.262478e+02,0.000166,2021-03-27 08:16:29
6,1.378062e+07,1857.350311,8.585947e+06,1.532300,2021-04-20 20:56:53
7,1.273397e+02,0.017883,1.225493e+02,0.000014,2021-05-04 10:54:47
8,1.511696e+03,0.318917,2.086123e+03,0.000209,2021-05-05 06:14:59
9,1.912582e+03,0.364567,2.542336e+03,0.000251,2021-05-07 21:46:23


### Writing swaps, mints, burns, reserves into pickle files

In [58]:
doge_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_swaps.pkl")
doge_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_reserves.pkl")
doge_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_mints.pkl")
doge_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [59]:
doge_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_swaps.pkl")
doge_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_reserves.pkl")
doge_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_mints.pkl")
doge_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_burns.pkl")

# ELON/ETH pool

In [ ]:
elon_weth_history = get_pool_v2_history(contract_id='0x7b73644935b8e68019ac6356c40661e1bc315860', range_limit=297)
elon_weth_df = pool_history_to_df(elon_weth_history)
elon_weth_df

In [66]:
elon_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x7b73644935b8e68019ac6356c40661e1bc315860')
elon_weth_reserves_df = pool_reserves_to_df(elon_weth_reserves_history)
elon_weth_reserves_df

100%|██████████| 2/2 [00:02<00:00,  1.16s/it]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,3.650502e+13,715.602627,3.385606e+06,2.729964e+15,41193.755814,2021-04-23
1,2.459976e+13,1110.292501,4.916181e+06,5.911238e+14,22876.541746,2021-04-24
2,2.935363e+13,933.446347,4.311324e+06,1.858092e+14,6450.296998,2021-04-25
3,2.492029e+13,1127.543829,5.708305e+06,2.802904e+14,12044.298435,2021-04-26
4,1.801824e+13,1632.782457,8.678267e+06,1.908413e+14,13764.348242,2021-04-27
...,...,...,...,...,...,...
198,1.128041e+13,5113.576467,4.708419e+07,3.279026e+13,14488.690623,2021-11-07
199,1.187093e+13,4873.017955,4.682281e+07,1.107740e+13,4677.153491,2021-11-08
200,1.233480e+13,4701.308475,4.453359e+07,1.016520e+13,3846.178431,2021-11-09
201,1.267539e+13,4588.670503,4.241485e+07,1.286510e+13,4489.571770,2021-11-10


In [67]:
elon_weth_mints_history = get_pool_v2_mints(contract_id='0x7b73644935b8e68019ac6356c40661e1bc315860', range_limit=2)
elon_weth_mints_df = pool_mints_to_df(elon_weth_mints_history)
elon_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  2.15it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,5.000000e+14,40.000000,94891.657784,1.414214e+08,2021-04-23 01:09:59
1,3.147686e+10,0.120000,536.883493,6.074555e+04,2021-04-23 04:19:58
2,2.215014e+10,0.133060,585.502662,5.357215e+04,2021-04-23 05:09:10
3,5.912952e+10,1.000000,4408.241274,2.386108e+05,2021-04-23 06:58:15
4,1.477064e+10,0.261188,1151.919743,6.094134e+04,2021-04-23 07:01:49
...,...,...,...,...,...
446,5.708324e+08,0.236869,2122.665970,6.972014e+03,2021-11-05 12:46:00
447,9.999952e+07,0.037955,335.920786,1.164084e+03,2021-11-06 17:16:15
448,5.242781e+09,2.464956,22698.393388,6.762486e+04,2021-11-07 19:29:59
449,2.068856e+07,0.008000,76.124019,2.412962e+02,2021-11-09 17:41:02


In [68]:
elon_weth_burns_history = get_pool_v2_burns(contract_id='0x7b73644935b8e68019ac6356c40661e1bc315860', range_limit=2)
elon_weth_burns_df = pool_burns_to_df(elon_weth_burns_history)
elon_weth_burns_df

100%|██████████| 2/2 [00:00<00:00,  2.31it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1.832062e+10,0.212365,920.838058,60941.336199,2021-04-23 07:33:39
1,4.605134e+09,0.212919,970.975245,30021.200665,2021-04-23 12:22:04
2,9.950712e+09,0.411064,1859.034736,60745.553498,2021-04-23 14:24:50
3,2.904929e+10,1.225029,5664.795677,178166.748472,2021-04-23 17:10:29
4,1.763913e+09,0.106315,499.053989,12850.654109,2021-04-23 18:47:15
...,...,...,...,...,...
327,5.789237e+08,0.233597,2097.224671,6972.013754,2021-11-05 13:18:58
328,4.286501e+08,0.165774,1487.560301,5043.639947,2021-11-06 07:27:16
329,1.286917e+09,0.455762,4082.459085,14486.498610,2021-11-06 08:58:34
330,6.567017e+07,0.026468,240.752819,786.481800,2021-11-07 08:29:47


### Writing swaps, mints, burns, reserves into pickle files

In [70]:
elon_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_swaps.pkl")
elon_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_reserves.pkl")
elon_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_mints.pkl")
elon_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [71]:
elon_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_swaps.pkl")
elon_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_reserves.pkl")
elon_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_mints.pkl")
elon_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_burns.pkl")

# SHIB/ETH pool

In [ ]:
shib_weth_history = get_pool_v2_history(contract_id='0x811beed0119b4afce20d2583eb608c6f7af1954f', range_limit=1205)
shib_weth_df = pool_history_to_df(shib_weth_history)
shib_weth_df

In [78]:
shib_weth_df

,token_in,token_out,amount_in,amount_out,amount_usd,timestamp
0,WETH,SHIB,4.000000e-01,1.898354e+13,139.381551,2020-07-31 20:42:11
1,WETH,SHIB,4.000000e-01,1.757930e+13,139.276591,2020-07-31 20:42:36
2,WETH,SHIB,2.000000e+00,7.144941e+13,746.446085,2020-07-31 20:43:55
3,WETH,SHIB,1.000000e-01,2.990975e+12,34.282991,2020-07-31 20:44:34
4,WETH,SHIB,1.000000e+00,2.320679e+13,353.606470,2020-07-31 20:45:35
...,...,...,...,...,...,...
1204940,WETH,SHIB,1.482560e-01,1.256766e+07,703.363025,2021-11-11 15:23:31
1204941,SHIB,WETH,5.000000e+07,5.856120e-01,2778.307967,2021-11-11 15:23:59
1204942,SHIB,WETH,4.451439e+07,5.218198e-01,2475.659643,2021-11-11 15:23:59
1204943,SHIB,WETH,1.267789e+07,1.484002e-01,704.116159,2021-11-11 15:26:01


In [73]:
shib_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x811beed0119b4afce20d2583eb608c6f7af1954f')
shib_weth_reserves_df = pool_reserves_to_df(shib_weth_reserves_history)
shib_weth_reserves_df

100%|██████████| 2/2 [00:03<00:00,  1.91s/it]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,3.475561e+13,149.972185,1.031592e+05,1.391419e+15,1092.291376,2020-07-31
1,4.995162e+13,128.090506,9.822113e+04,1.450646e+15,4047.355659,2020-08-01
2,7.443586e+13,103.836139,7.637526e+04,3.419726e+14,540.039674,2020-08-02
3,1.394962e+14,57.242026,4.402689e+04,4.040179e+14,341.670235,2020-08-03
4,1.390480e+14,58.146782,4.547281e+04,1.100752e+14,48.841749,2020-08-04
...,...,...,...,...,...,...
431,1.048962e+11,1281.695359,1.179915e+07,2.647354e+11,3283.003701,2021-11-07
432,1.090860e+11,1239.428833,1.190776e+07,2.260270e+11,2599.167270,2021-11-08
433,1.078935e+11,1250.448975,1.185011e+07,1.207557e+11,1422.022620,2021-11-09
434,1.113709e+11,1214.670010,1.122766e+07,2.489593e+11,2653.751122,2021-11-10


In [74]:
shib_weth_mints_history = get_pool_v2_mints(contract_id='0x811beed0119b4afce20d2583eb608c6f7af1954f', range_limit=2)
shib_weth_mints_df = pool_mints_to_df(shib_weth_mints_history)
shib_weth_mints_df

100%|██████████| 2/2 [00:02<00:00,  1.08s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,4.950000e+14,1.000000e+01,3.421447e+03,7.035624e+07,2020-07-31 20:42:11
1,2.385578e+11,4.610574e-01,3.156094e+02,3.258480e+05,2020-07-31 23:28:57
2,5.078508e+11,2.534585e+00,1.738039e+03,1.107577e+06,2020-08-01 00:28:52
3,2.850964e+12,6.645802e+00,4.552039e+03,4.205740e+06,2020-08-01 01:38:39
4,1.905753e+11,3.435528e-01,2.355079e+02,2.460249e+05,2020-08-01 02:06:23
...,...,...,...,...,...
1995,4.454220e+07,1.400000e-01,7.668379e+02,7.079650e+02,2021-06-07 00:27:04
1996,1.000000e+08,3.148006e-01,1.726741e+03,1.590509e+03,2021-06-07 00:37:47
1997,1.059140e+07,3.303798e-02,1.821972e+02,1.676571e+02,2021-06-07 01:23:25
1998,2.187981e+08,6.794273e-01,3.791229e+03,3.454987e+03,2021-06-07 02:19:48


In [75]:
shib_weth_burns_history = get_pool_v2_burns(contract_id='0x811beed0119b4afce20d2583eb608c6f7af1954f', range_limit=2)
shib_weth_burns_df = pool_burns_to_df(shib_weth_burns_history)
shib_weth_burns_df

100%|██████████| 2/2 [00:02<00:00,  1.21s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,2.680203e+11,0.427597,294.125991,3.258480e+05,2020-08-01 01:53:35
1,2.566214e+12,7.721197,5482.467576,4.205740e+06,2020-08-01 10:11:37
2,7.284586e+11,1.888833,1325.914127,1.107577e+06,2020-08-01 11:33:36
3,1.907353e+11,0.389869,279.675095,2.568436e+05,2020-08-01 16:35:39
4,7.526120e+10,0.157526,113.293753,1.025480e+05,2020-08-01 16:45:55
...,...,...,...,...,...
1995,1.037841e+08,0.382732,1552.068697,1.525532e+03,2021-07-13 07:44:22
1996,2.715608e+06,0.009903,39.779427,3.969099e+01,2021-07-13 11:51:48
1997,4.053543e+07,0.147503,583.155949,5.916668e+02,2021-07-13 14:45:25
1998,8.424326e+06,0.030611,121.975048,1.228624e+02,2021-07-13 17:34:31


### Writing swaps, mints, burns, reserves into pickle files

In [79]:
shib_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_swaps.pkl")
shib_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_reserves.pkl")
shib_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_mints.pkl")
shib_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [80]:
shib_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_swaps.pkl")
shib_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_reserves.pkl")
shib_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_mints.pkl")
shib_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_burns.pkl")